# Optimizing the model with series of experiments

In [1]:
import pathlib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

### Let´s first reload the data from the previous notebook. Then, perform train_test_split once again.

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

/home/arthur/Documentos/Insper/6_semestre/projetos_ml/AmesHousingDataset/data


In [3]:
model_data_scaled_path = DATA_DIR / 'processed' / 'ames_model_data_scaled.pkl'

In [4]:
data = pd.read_pickle(model_data_scaled_path)

In [5]:
X = data.drop(columns=['SalePrice']).copy().values
y = data['SalePrice'].copy().values

In [6]:
X.shape, y.shape

((2877, 164), (2877,))

In [7]:
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

### Experiment 1: LinearRegression with encoded categorical values AND scaled continuous variables

Our first experiment is to rerun the linear regression model with the same features as before, but this time on the new data with scaled continuous variables. 

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
linear_scaled_model = LinearRegression()

linear_scaled_model.fit(Xtrain, ytrain)

LinearRegression()

In [14]:
scores_linear_scaled = cross_val_score(linear_scaled_model,
                                       Xtrain, 
                                       ytrain,
                                       cv=10,
                                       scoring='neg_mean_squared_error',
                                       n_jobs=-1)

scores_linear_scaled = np.sqrt(-scores_linear_scaled)
print(f'MEAN: {scores_linear_scaled.mean():.2f} +/- {scores_linear_scaled.std():.2f}')
error_percent_linear = 100 * (10**scores_linear_scaled.mean() - 1)
print(f'Average error is {error_percent_linear:.2f}%')

MEAN: 0.05 +/- 0.01
Average error is 12.77%


The experiment showed no signifficant difference in the model performance. This means that the LinearRegression model is not sensitive to the scale of the continuous variables. We will, however, still keep the scaled data for the next experiments.

### Experiment 2: Lasso Regression

In [15]:
from sklearn.linear_model import Lasso

In [22]:
lasso = Lasso()

params = {
    'alpha': np.logspace(-4, 0, 100)
}

grid = GridSearchCV(lasso, params, cv=10, verbose=1, n_jobs=-1)

In [23]:
grid.fit(Xtrain, ytrain)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.210e-02, tolerance: 5.590e-03
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=10, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-04, 1.09749877e-04, 1.20450354e-04, 1.32194115e-04,
       1.45082878e-04, 1.59228279e-04, 1.74752840e-04, 1.91791026e-04,
       2.10490414e-04, 2.31012970e-04, 2.53536449e-04, 2.78255940e-04,
       3.05385551e-04, 3.35160265e-04, 3.67837977e-04, 4.03701726e-04,
       4.43062146e-04, 4.86260158e-04, 5.3366...
       1.17681195e-01, 1.29154967e-01, 1.41747416e-01, 1.55567614e-01,
       1.70735265e-01, 1.87381742e-01, 2.05651231e-01, 2.25701972e-01,
       2.47707636e-01, 2.71858824e-01, 2.98364724e-01, 3.27454916e-01,
       3.59381366e-01, 3.94420606e-01, 4.32876128e-01, 4.75081016e-01,
       5.21400829e-01, 5.72236766e-01, 6.28029144e-01, 6.89261210e-01,
       7.56463328e-01, 8.30217568e-01, 9.11162756e-01, 1.00000000e+00])},
             verbose=1)

In [26]:
best_params = grid.best_params_
print(best_params)

{'alpha': 0.0001}


In [27]:
lasso_best = Lasso(**best_params)
lasso_best.fit(Xtrain, ytrain)

Lasso(alpha=0.0001)

In [28]:
scores_lasso = cross_val_score(lasso_best, Xtrain, ytrain, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
scores_lasso = np.sqrt(-scores_lasso)
print(f'MEAN: {scores_lasso.mean():.2f} +/- {scores_lasso.std():.2f}')
error_percent_lasso = 100 * (10**scores_lasso.mean() - 1)
print(f'Average error is {error_percent_lasso:.2f}%')

MEAN: 0.05 +/- 0.01
Average error is 13.16%


### Experiment 3:  Random Forest Regression

In [30]:
from sklearn.ensemble import RandomForestRegressor

In [51]:
rf = RandomForestRegressor()

params = {
    'n_estimators': [100,125,150,200],
    'min_samples_split': [3],
    'max_features': ['auto', 'sqrt'],
    'verbose': [0, 1, 2, 3],
    'max_depth': [None],
    'bootstrap': [False],
    'min_weight_fraction_leaf': [0.0],
    'min_samples_leaf': [1],
}

grid = GridSearchCV(rf, params, cv=5, verbose=1, n_jobs=-1)

In [52]:
grid.fit(Xtrain, ytrain)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 1 of 100
building tree 29 of 100
building tree 2 of 100
building tree 30 of 100
building tree 3 of 100
building tree 31 of 100
building tree 32 of 100
building tree 4 of 100
building tree 33 of 100
building tree 5 of 100
building tree 34 of 100
building tree 35 of 100
building tree 6 of 100
building tree 36 of 100
buildin

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent wor


building tree 78 of 100
building tree 36 of 100
building tree 3 of 100
building tree 11 of 100
building tree 21 of 100
building tree 79 of 100
building tree 37 of 100
building tree 4 of 100
building tree 12 of 100
building tree 80 of 100
building tree 22 of 100
building tree 38 of 100
building tree 5 of 100
building tree 13 of 100
building tree 81 of 100
building tree 23 of 100
building tree 39 of 100
building tree 6 of 100
building tree 14 of 100
building tree 82 of 100
building tree 24 of 100
building tree 40 of 100
building tree 7 of 100
building tree 15 of 100
building tree 83 of 100
building tree 25 of 100
building tree 41 of 100
building tree 8 of 100
building tree 16 of 100
building tree 84 of 100
building tree 1 of 100
building tree 26 of 100
building tree 42 of 100
building tree 9 of 100
building tree 17 of 100
building tree 85 of 100
building tree 2 of 100
building tree 27 of 100
building tree 10 of 100
building tree 3 of 100
building tree 18 of 100
building tree 11 of 100
b

...

building tree 181 of 200
building tree 178 of 200
building tree 169 of 200
building tree 179 of 200
building tree 182 of 200
building tree 170 of 200
building tree 180 of 200
building tree 183 of 200
building tree 171 of 200
building tree 181 of 200
building tree 184 of 200
building tree 182 of 200
building tree 172 of 200
building tree 185 of 200
building tree 183 of 200
building tree 173 of 200
building tree 184 of 200
building tree 186 of 200
building tree 174 of 200
building tree 185 of 200
building tree 187 of 200
building tree 186 of 200
building tree 175 of 200
building tree 187 of 200
building tree 188 of 200
building tree 176 of 200
building tree 188 of 200
building tree 189 of 200
building tree 177 of 200
building tree 189 of 200
building tree 190 of 200
building tree 190 of 200
building tree 178 of 200
building tree 191 of 200
building tree 191 of 200
building tree 179 of 200
building tree 192 of 200
building tree 193 of 200
building tree 192 of 200
building tree 180 of 200


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
80 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
43 fits failed with the following error:
Traceback (most recent call last):
  File "/home/arthur

building tree 199 of 200
building tree 200 of 200


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1], 'min_samples_split': [3],
                         'min_weight_fraction_leaf': [0.0],
                         'n_estimators': [100, 125, 150, 200],
                         'verbose': [0, 1, 2, 3]},
             verbose=1)

In [53]:
best_params = grid.best_params_
print(best_params)

{'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 3, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'verbose': 1}


In [54]:
rf_best = RandomForestRegressor(**best_params)
rf_best.fit(Xtrain, ytrain)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


RandomForestRegressor(bootstrap=False, max_features='sqrt', min_samples_split=3,
                      verbose=1)

In [55]:
scores_rf = cross_val_score(rf_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_rf = np.sqrt(-scores_rf)
print(f'MEAN: {scores_rf.mean():.2f} +/- {scores_rf.std():.2f}')
error_percent_rf = 100 * (10**scores_rf.mean() - 1)
print(f'Average error is {error_percent_rf:.2f}%')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 1

MEAN: 0.05 +/- 0.01
Average error is 13.37%


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


### Experiment 4:  Gradient Boosting Regression

In [56]:
from sklearn.ensemble import GradientBoostingRegressor

In [59]:
gbr = GradientBoostingRegressor()

params = {
    'learning_rate': [0.1, 0.05],
    'n_estimators': [100, 150],
    'subsample': [1, 0.9],
    'criterion': ['friedman_mse', 'mse'],
    'min_samples_split': [2,3],
    'min_samples_leaf': [1,2],
    'max_depth': [3,4],
    'min_impurity_decrease': [0, 0.1],
    'max_features': ['auto', 'sqrt'],
    'alpha': [0.9, 0.8],
    'verbose': [0, 1],
    'warm_start': [True, False],
}

grid = GridSearchCV(gbr, params, cv=5, verbose=1, n_jobs=-1)

In [60]:
grid.fit(Xtrain, ytrain)

Fitting 5 folds for each of 4096 candidates, totalling 20480 fits


      Iter       Train Loss   Remaining Time 
         1           0.0263            0.16s
         2           0.0234            0.15s
         3           0.0207            0.16s
         4           0.0184            0.21s
         5           0.0166            0.21s
         6           0.0151            0.22s
         7           0.0135            0.21s
         8           0.0121            0.21s
         9           0.0110            0.20s
        10           0.0101            0.21s
        20           0.0052            0.19s
      Iter       Train Loss   Remaining Time 
         1           0.0255            0.26s
         2           0.0231            0.23s
         3           0.0209            0.21s
        30           0.0036            0.16s
         4           0.0192            0.20s
         5           0.0174            0.22s
         6           0.0159            0.24s
         7           0.0145            0.24s
         8           0.0135            0.23s
        

/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
15360 fits failed out of a total of 20480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2986 fits failed with the following error:
Traceback (most recent call last):
  File "/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/home/arthur/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "/ho

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.0260           0.0031            0.39s
         2           0.0224           0.0011            0.44s
         3           0.0198           0.0014            0.40s
         4           0.0174           0.0032            0.40s
         5           0.0155           0.0006            0.37s
         6           0.0139           0.0021            0.34s
         7           0.0123           0.0003            0.32s
         8           0.0112           0.0029            0.31s
         9           0.0101           0.0017            0.32s
        10           0.0089           0.0002            0.33s
        20           0.0043           0.0009            0.30s
        30           0.0028           0.0000            0.26s
        40           0.0022          -0.0001            0.24s
        50           0.0019          -0.0003            0.22s
        60           0.0017           0.0003            0.20s
       

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'alpha': [0.9, 0.8],
                         'criterion': ['friedman_mse', 'mse'],
                         'learning_rate': [0.1, 0.05], 'max_depth': [3, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_impurity_decrease': [0, 0.1],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3],
                         'n_estimators': [100, 150], 'subsample': [1, 0.9],
                         'verbose': [0, 1], 'warm_start': [True, False]},
             verbose=1)

In [61]:
best_params = grid.best_params_
print(best_params)

{'alpha': 0.9, 'criterion': 'friedman_mse', 'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_impurity_decrease': 0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 150, 'subsample': 0.9, 'verbose': 1, 'warm_start': True}


In [62]:
gbr_best = GradientBoostingRegressor(**best_params)
gbr_best.fit(Xtrain, ytrain)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.0257           0.0038            0.44s
         2           0.0223           0.0011            0.35s
         3           0.0193           0.0003            0.35s
         4           0.0173           0.0060            0.33s
         5           0.0152           0.0007            0.32s
         6           0.0133          -0.0008            0.31s
         7           0.0121           0.0051            0.29s
         8           0.0111           0.0013            0.28s
         9           0.0100           0.0008            0.28s
        10           0.0088          -0.0014            0.27s
        20           0.0042           0.0008            0.20s
        30           0.0028          -0.0008            0.19s
        40           0.0023           0.0009            0.16s
        50           0.0020          -0.0003            0.14s
        60           0.0017          -0.0002            0.13s
       

GradientBoostingRegressor(max_depth=4, max_features='sqrt',
                          min_impurity_decrease=0, min_samples_leaf=2,
                          n_estimators=150, subsample=0.9, verbose=1,
                          warm_start=True)

In [63]:
scores_gbr = cross_val_score(gbr_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_gbr = np.sqrt(-scores_gbr)
print(f'MEAN: {scores_gbr.mean():.2f} +/- {scores_gbr.std():.2f}')
error_percent_gbr = 100 * (10**scores_gbr.mean() - 1)
print(f'Average error is {error_percent_gbr:.2f}%')

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.0262           0.0028            0.22s
         2           0.0224          -0.0001            0.24s
         3           0.0196           0.0009            0.24s
         4           0.0180           0.0060            0.26s
         5           0.0161           0.0032            0.25s
         6           0.0140          -0.0010            0.25s
         7           0.0124           0.0019            0.27s
         8           0.0112           0.0018            0.27s
         9           0.0099          -0.0019            0.26s
        10           0.0094           0.0047            0.25s
        20           0.0042          -0.0000            0.24s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.0261           0.0034            0.29s
         2           0.0232           0.0045            0.29s
         3           0.0206           0.0050            0.27s
      

### Experiment 5:  KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()

params = {
    'n_neighbors': [5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'p': [1, 2, 3],
}

grid = GridSearchCV(knn, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
knn_best = KNeighborsRegressor(**best_params)
knn_best.fit(Xtrain, ytrain)

In [ ]:
y_pred_knn = knn_best.predict(Xtest)

In [ ]:
RMSE = np.sqrt(mean_squared_error(ytest, y_pred_knn))
error_percent = 100 * (10**RMSE - 1)
print(f'RMSE: {RMSE}')
print(f'Average error is {error_percent:.2f}%')

### Experiment 6:  Regression Stack

### Retraining of best model on the whole dataset

In [ ]:
best_model = linear_scaled_model

In [ ]:
best_model.fit(X, y)